<a href="https://colab.research.google.com/github/fernandabarrigosse/LIA/blob/main/RAG_STJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instalando as dependecias

In [54]:
!pip install pymupdf langchain sentence-transformers faiss-cpu transformers torch

In [55]:
import fitz
import re
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer
import torch
import os
import json

Extrair sumulas do pdf

In [56]:
import json

def extrair_sumulas(pdf_path):
    sumulas = []
    text_buffer = ""

    # Abrir o PDF com PyMuPDF
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc[page_num]
        text_buffer += page.get_text("text") + "\n"

        # Processar o buffer a cada 100 páginas para reduzir memória
        if page_num % 100 == 0 or page_num == len(doc) - 1:
            padrao = r'SÚMULA\s+(\d+)\s*\n\s*([^\n]+?)\s*\n\s*(?:Enunciado:)?\s*\n(.*?)(?=\s*(?:SÚMULA\s+\d+|INTEIRO TEOR DAS SÚMULAS|Referências Legislativas:|Anexos|\Z))'
            matches = re.findall(padrao, text_buffer, re.DOTALL | re.IGNORECASE)

            for match in matches:
                numero = match[0]
                area_titulo = match[1].strip()
                enunciado = match[2].strip()
                sumulas.append({
                    "numero": numero,
                    "area_titulo": area_titulo,
                    "enunciado": enunciado
                })

            # Limpar buffer
            text_buffer = ""

    doc.close()

    output_dir = '/content'

    with open(os.path.join(output_dir, 'sumulas_completas.json'), 'w', encoding='utf-8') as f:
        json.dump(sumulas, f, ensure_ascii=False, indent=4)


    return sumulas

In [57]:
def criar_vector_store(sumulas, index_path="/content/sumulas_index"):
    embeddings = HuggingFaceEmbeddings(model_name="neuralmind/bert-base-portuguese-cased")
    texts = [f"Súmula {s['numero']} - {s['area_titulo']}: {s['enunciado']}" for s in sumulas]
    metadatas = [{"numero": s["numero"], "area_titulo": s["area_titulo"]} for s in sumulas]

    vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
    vectorstore.save_local(index_path)
    return vectorstore

In [58]:
def configurar_rag(vectorstore):
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    text_generation_pipeline = pipeline(
        "text2text-generation",
        model=model_name,
        tokenizer=tokenizer,
        max_length=512,
        device=0 if torch.cuda.is_available() else -1
    )
    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )
    return qa_chain, tokenizer

In [59]:
def verificar_ementa(qa_chain, tokenizer, ementa):
    prompt = f"""
    Verifique se existe alguma súmula do STJ em consonância com a ementa abaixo. Considere o alinhamento jurídico, incluindo princípios e legislações citadas. Liste as súmulas relevantes, explique o raciocínio e indique se há enquadramento total, parcial ou nenhum. Se nenhuma súmula for aplicável, explique por quê.

    Ementa: {ementa}
    """
    # Truncar prompt para 400 tokens
    tokens = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)
    truncated_prompt = tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)

    resultado = qa_chain.invoke({"query": truncated_prompt})
    return resultado["result"], resultado["source_documents"]

In [60]:
pdf_path = "/content/SumulasSTJ.pdf"
sumulas = extrair_sumulas(pdf_path)
print(f"Extraídas {len(sumulas)} súmulas")

Extraídas 555 súmulas


In [61]:
index_path = "/content/sumulas_index"
if os.path.exists(index_path):
    vectorstore = FAISS.load_local(index_path, HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"), allow_dangerous_deserialization=True)
else:
    vectorstore = criar_vector_store(sumulas, index_path)

qa_chain, tokenizer = configurar_rag(vectorstore)

Device set to use cpu


In [63]:
ementa = """
PRISÃO PREVENTIVA. PORTE ILEGAL DE ARMA DE FOGO DE USO PERMITIDO, TRÁFICO DE DROGAS E ASSOCIAÇÃO PARA O TRÁFICO. GARANTIA DA ORDEM PÚBLICA. CONVERSÃO EX OFFICIO DA PRISÃO EM FLAGRANTE EM PREVENTIVA. IMPOSSIBILIDADE. NECESSIDADE DE REQUERIMENTO PRÉVIO OU PELO MINISTÉRIO PÚBLICO OU PELO QUERELANTE, OU PELO ASSISTENTE OU, POR FIM, MEDIANTE REPRESENTAÇÃO DA AUTORIDADE POLICIAL. [...] No caso, a decisão agravada deve ser mantida, uma vez que não é possível a decretação da prisão preventiva de ofício em face do que dispõe a Lei n. 13.964/2019, mesmo se decorrente de prisão em flagrante e se não tiver ocorrido audiência de custódia. Isso porque não existe diferença entre a conversão da prisão em flagrante em preventiva e a decretação da prisão preventiva como uma primeira prisão (EDcl no AgRg no HC n. 653.425/MG, de minha relatoria, Sexta Turma, DJe 19/11/2021)
"""
resposta, documentos = verificar_ementa(qa_chain, tokenizer, ementa)
print("Resposta:", resposta)
print("Súmulas recuperadas:", [doc.metadata for doc in documentos])

Resposta: EX OFFICIO DA PRISO EM FLAGRANTE EM PREVENTIVA. IMPOSSIBILIDADE. NECESSIDADE DE REQUERIMENTO PRÉVIO OU PELO MINISTÉRIO PBLICO OU PELO QUERELANTE, OU PELO ASSISTENTE OU, POR FIM, MEDIANTE REPRESENTAO DA AUTORIDADE POLICIAL
Súmulas recuperadas: [{'numero': '213', 'area_titulo': 'DIREITO TRIBUTÁRIO - COMPENSAÇÃO DE CRÉDITOS TRIBUTÁRIOS'}, {'numero': '191', 'area_titulo': 'DIREITO PENAL - PRESCRIÇÃO'}, {'numero': '12', 'area_titulo': 'DIREITO ADMINISTRATIVO - DESAPROPRIAÇÃO'}]
